<a href="https://colab.research.google.com/github/satyamverma95/DL_NLP/blob/main/Pre_requisite_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [3]:
#!pip install evaluate
#!pip install huggingface_hub
#!pip install datasets
#!pip install transformers==4.28.0
#!pip install --upgrade accelerate

Getting Token authentication cell and input your username and password:

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers

print(transformers.__version__)

4.28.0


# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).

![Widget inference on a text classification task](https://github.com/huggingface/notebooks/blob/main/examples/images/text_classification.png?raw=1)

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [47]:
task = "rte"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
metric = load_metric('glue', task)

<ipython-input-47-681dac30ef8e>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', task)


The DistilBERT model has several pre-trained variants with different configurations. As of my knowledge cutoff in September 2021, the following DistilBERT models were available:

1. `distilbert-base-uncased`: The base variant of DistilBERT with an uncased vocabulary.
2. `distilbert-base-uncased-distilled-squad`: A variant of DistilBERT fine-tuned on the Stanford Question Answering Dataset (SQuAD). It is specifically trained for question answering tasks.
3. `distilbert-base-cased`: Similar to `distilbert-base-uncased`, but with a cased vocabulary.
4. `distilbert-base-cased-distilled-squad`: Similar to `distilbert-base-uncased-distilled-squad`, but with a cased vocabulary.
5. `distilbert-base-multilingual-cased`: A variant of DistilBERT trained on a multilingual corpus with a cased vocabulary. It supports multiple languages.
6. `distilbert-base-german-cased`: A variant of DistilBERT trained specifically for the German language with a cased vocabulary.

Please note that new variants of DistilBERT may have been introduced since my knowledge cutoff. To get the most up-to-date information on available DistilBERT models, I recommend referring to the Hugging Face Model Hub or the official DistilBERT repository.

## Loading the dataset

Mounting the Google drive Content Folder.
Uploading the file to Content Folder.

In [6]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [7]:
csv_file_path = '/content/Pre-requisite_dataset#2.csv'

In [8]:
from datasets import DatasetDict, Dataset, load_dataset, load_metric
dataset = Dataset.from_csv(csv_file_path)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8bbf91ee7443d1a1/0.0.0. Subsequent calls will reuse this data.


In [9]:
dataset

Dataset({
    features: ['Topic', 'Prerequsites', 'label'],
    num_rows: 12960
})

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [10]:
#actual_task = "mnli" if task == "mnli-mm" else task
#dataset = load_dataset("glue", actual_task)
#metric = load_metric('glue', actual_task)
dataset = dataset.train_test_split(test_size=0.2)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

To access an actual element, you need to select a split first, then give an index:

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Topic', 'Prerequsites', 'label'],
        num_rows: 10368
    })
    test: Dataset({
        features: ['Topic', 'Prerequsites', 'label'],
        num_rows: 2592
    })
})

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [12]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(dataset["train"])

,Topic,Prerequsites,label
0,Online parameter learning using recursive least squares,Factorial HMM,1
1,KL divergence,The multivariate Gaussian,0
2,Online parameter learning using recursive least squares,Generalizations of HMMs,0
3,Singular value decomposition (SVD),Mixtures of factor analysers,1
4,Maximum likelihood estimation,Independent Component Analysis (ICA),1
5,Generalized linear models (GLMs),Residual analysis (outlier detection),0
6,"Approximate online inference for non-linear, non-Gaussian SSMs",Learning for LG-SSM,1
7,Inference in jointly Gaussian distributions,Classifying documents using bag of words,0
8,Example: factorial HMM,Message passing on a junction tree,0
9,Markov chain Monte Carlo (MCMC) inference,RBPF for switching LG-SSMs,1


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [14]:
from transformers import AutoTokenizer
from transformers import BertTokenizer    

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [16]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("Topic", "Prerequsites"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [17]:
sentence1_key, sentence2_key = task_to_keys[task]
print("sentence1_key :{}, sentence2_key:{}".format(sentence1_key, sentence2_key))
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Topic: {dataset['train'][0][sentence1_key]}")
    print(f"Prerequsites: {dataset['train'][0][sentence2_key]}")

sentence1_key :Topic, sentence2_key:Prerequsites
Topic: Subspace methods
Prerequsites: Inference in LG-SSM


We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [18]:
def preprocess_function(examples):
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation='do_not_truncate', max_length=100, padding="max_length")

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [19]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[101, 24807, 10732, 4725, 102, 28937, 1999, 1048, 2290, 1011, 7020, 2213, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 4646, 1024, 6346, 11616, 102, 28937, 1999, 1048, 2290, 1011, 7020, 2213, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [20]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10368 [00:00<?, ? examples/s]

Map:   0%|          | 0/2592 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [21]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-mrpc"` or `"huggingface/bert-finetuned-mrpc"`).

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [61]:
import numpy as np
from sklearn.metrics import accuracy_score


def compute_accuracy(eval_pred):
    print("Eval Predict", eval_pred)
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    print("predictions : {}, labels : {}".format(predictions, labels))
    accuracy = accuracy_score(labels, predictions)
    print ("Accuracy : {}".format(accuracy))
    return {"accuracy": accuracy}

In [74]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #print("Eval", eval_pred)
    #print("predictions : {}, labels : {}".format(predictions, labels))
    predictions = np.argmax(predictions, axis=1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    print ("predictions :{}, accuracy : {}".format(predictions, accuracy))
    return (accuracy)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

/content/distilbert-base-uncased-finetuned-rte is already a clone of https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-rte. Make sure you pull the latest changes with `repo.git_pull()`.


You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [23]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [75]:
training_args = TrainingArguments(
    output_dir="Pre-requisite_Model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/Pre-requisite_Model is already a clone of https://huggingface.co/satyamverma/Pre-requisite_Model. Make sure you pull the latest changes with `repo.git_pull()`.


In [53]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.348000,0.737006,0.665509
2,0.341100,0.737006,0.665509
3,0.339400,0.737006,0.665509


Eval <transformers.trainer_utils.EvalPrediction object at 0x7f7bf1c1f5b0>
Eval <transformers.trainer_utils.EvalPrediction object at 0x7f7b98429090>
Eval <transformers.trainer_utils.EvalPrediction object at 0x7f7ab3db8c40>


TrainOutput(global_step=1944, training_loss=0.3436668929739744, metrics={'train_runtime': 326.1642, 'train_samples_per_second': 95.363, 'train_steps_per_second': 5.96, 'total_flos': 804739446835200.0, 'train_loss': 0.3436668929739744, 'epoch': 3.0})

In [26]:
trainer

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [76]:
trainer.evaluate()

predictions :[1 1 0 ... 1 0 0], accuracy : {'accuracy': 0.6655092592592593}


{'eval_loss': 0.7370057106018066,
 'eval_accuracy': 0.6655092592592593,
 'eval_runtime': 6.6968,
 'eval_samples_per_second': 387.05,
 'eval_steps_per_second': 24.191}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

You can now upload the result of the training to the Hub, just execute this instruction:

In [77]:
trainer.push_to_hub()

To https://huggingface.co/satyamverma/Pre-requisite_Model
   e138a23..71d9377  main -> main

   e138a23..71d9377  main -> main

To https://huggingface.co/satyamverma/Pre-requisite_Model
   71d9377..71e08e8  main -> main

   71d9377..71e08e8  main -> main



'https://huggingface.co/satyamverma/Pre-requisite_Model/commit/71d9377f9e806b1da496845ed64fe5b5dfd431b2'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
```

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

In [79]:
#! pip install optuna
#! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

In [80]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [81]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10) 
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [82]:
best_run = trainer.hyperparameter_search(n_trials=6, direction="maximize")

[I 2023-05-23 06:08:29,802] A new study created in memory with name: no-name-a055294c-d558-4137-bfd1-3052dfcb62b6
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpo

Epoch,Training Loss,Validation Loss


[W 2023-05-23 06:09:18,751] Trial 0 failed with parameters: {'learning_rate': 1.8471891880522516e-05, 'num_train_epochs': 4, 'seed': 23, 'per_device_train_batch_size': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations.py", line 198, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1662, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2006, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2291, in _maybe_log_save_evaluate
    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2592 in hyperparameter_search    │
│                                                                                                  │
│   2589 │   │   │   HPSearchBackend.SIGOPT: run_hp_search_sigopt,                                 │
│   2590 │   │   │   HPSearchBackend.WANDB: run_hp_search_wandb,                                   │
│   2591 │   │   }                                                                                 │
│ ❱ 2592 │   │   best_run = backend_dict[backend](self, n_trials, direction, **kwargs)             │
│   2593 │   │                                                                                     │
│   2594 │   │   self.hp_search_backend = None                                                     │
│   2595 │   │   return best_run                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/integrations.py:208 in run_hp_search_optuna │
│                                                                                                  │
│    205 │   │   timeout = kwargs.pop("timeout", None)                                             │
│    206 │   │   n_jobs = kwargs.pop("n_jobs", 1)                                                  │
│    207 │   │   study = optuna.create_study(direction=direction, **kwargs)                        │
│ ❱  208 │   │   study.optimize(_objective, n_trials=n_trials, timeout=timeout, n_jobs=n_jobs)     │
│    209 │   │   best_trial = study.best_trial                                                     │
│    210 │   │   return BestRun(str(best_trial.number), best_trial.value, best_trial.params)       │
│    211 │   else:                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optuna/study/study.py:425 in optimize                    │
│                                                                                                  │
│    422 │   │   │   │   If nested invocation of this method occurs.                               │
│    423 │   │   """                                                                               │
│    424 │   │                                                                                     │
│ ❱  425 │   │   _optimize(                                                                        │
│    426 │   │   │   study=self,                                                                   │
│    427 │   │   │   func=func,                                                                    │
│    428 │   │   │   n_trials=n_trials,                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:66 in _optimize                │
│                                                                                                  │
│    63 │                                                                                          │
│    64 │   try:                                                                                   │
│    65 │   │   if n_jobs == 1:                                                                    │
│ ❱  66 │   │   │   _optimize_sequential(                                                          │
│    67 │   │   │   │   study,                               

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

BestRun(run_id='8', objective=0.631768953068592, hyperparameters={'learning_rate': 4.247277359513074e-05, 'num_train_epochs': 3, 'seed': 8, 'per_device_train_batch_size': 32}, run_summary=None)

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689807,0.581227
2,No log,0.665415,0.606498
3,No log,0.682738,0.631769


TrainOutput(global_step=234, training_loss=0.6310907999674479, metrics={'train_runtime': 195.6038, 'train_samples_per_second': 38.189, 'train_steps_per_second': 1.196, 'total_flos': 355213916749944.0, 'train_loss': 0.6310907999674479, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Upload file runs/May21_18-24-09_1d185abc4b07/events.out.tfevents.1684695850.1d185abc4b07.310.24: 100%|########…

To https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-rte
   054e621..300ba30  main -> main

   054e621..300ba30  main -> main

To https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-rte
   300ba30..8d7894b  main -> main

   300ba30..8d7894b  main -> main



'https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-rte/commit/300ba3066fd3d00cc53c1b443e450cd02edc4293'